<h1>Perform the fine-tuning on Llama2 model 🦙</h1>

In [2]:
import os
from huggingface_hub.hf_api import HfFolder
HfFolder.save_token(os.environ['HUGGING_FACE'])
!huggingface-cli whoami

shirastromer


In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/Users/shirabroner/Documents/DataScience/venv/Default/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [2]:
HF_REPO_NAME = 'shirastromer/pantry-ai-receipt-ocr-amount'

In [3]:
dataset = load_dataset(HF_REPO_NAME)
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2
    })
})

Copied from  [Guanaco-Llama - Dataset] notebook

In [4]:
dataset = dataset['train'].shuffle(seed=42).select(range(len(dataset) + 1))

# Define a function to transform the data
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}


# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)

In [5]:
transformed_dataset[0]

{'text': "<s>[INST] What is the weather today? [/INST] As an AI language model, I don't have access to real-time data such as current weather conditions </s>"}

In [6]:
dataset = load_dataset('timdettmers/openassistant-guanaco')

/Users/shirabroner/Documents/DataScience/venv/Default/lib/python3.9/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
